In [65]:
%pip install --user --break-system-packages langchain langgraph langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [66]:
%pip install --user --break-system-packages beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [67]:
%pip install --user --break-system-packages langchain-classic

Note: you may need to restart the kernel to use updated packages.


In [68]:
import functools
import operator
import requests
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from typing import TypedDict, Annotated, Sequence
from pydantic import BaseModel

load_dotenv('api.env')

True

In [69]:
llm = ChatOpenAI(model="gpt-4-turbo")

In [ ]:
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Used to fetch and process content from a valid URL on the internet. 
    The URL must be a complete, valid URL starting with http:// or https://.
    Example: https://www.example.com/article"""
    try:
        # Validate and format URL
        url = url.strip()
        if not url.startswith(('http://', 'https://')):
            return f"Error: Invalid URL format. URL must start with http:// or https://. Received: {url}"
        
        # Attempt to fetch the URL
        response = requests.get(url, timeout=10)
        response.raise_for_status()  
        
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text()
        
        if not text.strip():
            return "Warning: The page appears to be empty or contains no readable text."
        
        return text
    except requests.exceptions.RequestException as e:
        return f"Error: Failed to fetch URL '{url}'. {str(e)}. Please provide a valid, accessible URL."
    except Exception as e:
        return f"Error: An unexpected error occurred while processing the URL: {str(e)}"

tools = [process_search_tool]


In [71]:
def create_new_agent(llm: ChatOpenAI,
                  tools: list,
                  system_prompt: str) -> AgentExecutor:
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [72]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [73]:
content_marketing_team = ["online_researcher", "blog_manager", "social_media_manager"]

In [74]:
system_prompt = (
    "As a content marketing manager, your role is to oversee the insight between these"
    " workers: {content_marketing_team}. Based on the user's request,"
    " determine which worker should take the next action. Each worker is responsible for"
    " executing a specific task and reporting back thier findings and progress."
    " Once all tasks are completed, indicate 'FINISH'."
)

In [75]:
options = ["FINISH"] + content_marketing_team


from pydantic import BaseModel

class RouteDecision(BaseModel):
    """Select the next role."""
    next: str  

In [76]:
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}]}},
        "required": ["next"]
    }
}

In [77]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system",
     "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), content_marketing_team=", ".join(content_marketing_team))

In [78]:
content_marketing_manager_chain = (prompt | llm.with_structured_output(RouteDecision))

/usr/local/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:2067: UserWarning: Cannot use method='json_schema' with model gpt-4-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [79]:
online_researcher_agent = create_new_agent(
    llm,
    tools,
    """Your primary role is to function as an intelligent online research assistant, adept at scouring 
    the internet for the latest and most relevant trending stories across various sectors like politics, technology, 
    health, culture, and global events. You possess the capability to access a wide range of online news sources, 
    blogs, and social media platforms to gather real-time information."""
)

In [80]:
online_researcher_node = functools.partial(
    agent_node, agent=online_researcher_agent, name="online_researcher"
)

In [81]:
blog_manager_agent = create_new_agent(
    llm, tools,
    """You are a Blog Manager. The role of a Blog Manager encompasses several critical responsibilities aimed at transforming initial drafts into polished, SEO-optimized blog articles that engage and grow an audience. Starting with drafts provided by online researchers, the Blog Manager must thoroughly understand the content, ensuring it aligns with the blog's tone, target audience, and thematic goals. Key responsibilities include:

1. Content Enhancement: Elevate the draft's quality by improving clarity, flow, and engagement. This involves refining the narrative, adding compelling headers, and ensuring the article is reader-friendly and informative.

2. SEO Optimization: Implement best practices for search engine optimization. This includes keyword research and integration, optimizing meta descriptions, and ensuring URL structures and heading tags enhance visibility in search engine results.

3. Compliance and Best Practices: Ensure the content adheres to legal and ethical standards, including copyright laws and truth in advertising. The Blog Manager must also keep up with evolving SEO strategies and blogging trends to maintain and enhance content effectiveness.

4. Editorial Oversight: Work closely with writers and contributors to maintain a consistent voice and quality across all blog posts. This may also involve managing a content calendar, scheduling posts for optimal engagement, and coordinating with marketing teams to support promotional activities.

5. Analytics and Feedback Integration: Regularly review performance metrics to understand audience engagement and preferences. Use this data to refine future content and optimize overall blog strategy.

In summary, the Blog Manager plays a pivotal role in bridging initial research and the final publication by enhancing content quality, ensuring SEO compatibility, and aligning with the strategic objectives of the blog. This position requires a blend of creative, technical, and analytical skills to successfully manage and grow the blog's presence online.""")


blog_manager_node = functools.partial(
    agent_node, agent=blog_manager_agent, name="blog_manager")


social_media_manager_agent = create_new_agent(
    llm, tools,
    """You are a Social Media Manager. The role of a Social Media Manager, particularly for managing Twitter content, involves transforming research drafts into concise, engaging tweets that resonate with the audience and adhere to platform best practices. Upon receiving a draft from an online researcher, the Social Media Manager is tasked with several critical functions:

1. Content Condensation: Distill the core message of the draft into a tweet, which typically allows for only 280 characters. This requires a sharp focus on brevity while maintaining the essence and impact of the message.

2. Engagement Optimization: Craft tweets to maximize engagement. This includes the strategic use of compelling language, relevant hashtags, and timely topics that resonate with the target audience.

3. Compliance and Best Practices: Ensure that the tweets follow Twitter’s guidelines and best practices, including the appropriate use of mentions, hashtags, and links. Also, adhere to ethical standards, avoiding misinformation and respecting copyright norms.

In summary, the Social Media Manager's role is crucial in leveraging Twitter to disseminate information effectively, engage with followers, and build the brand’s presence online. This position combines creative communication skills with strategic planning and analysis to optimize social media impact.""")

social_media_manager_node = functools.partial(
    agent_node, agent=social_media_manager_agent, name="social_media_manager")

In [82]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

In [83]:
workflow = StateGraph(AgentState)

In [84]:
workflow.add_node("content_marketing_manager", content_marketing_manager_chain)
workflow.add_node("online_researcher", online_researcher_node)
workflow.add_node("blog_manager", blog_manager_node)
workflow.add_node("social_media_manager", social_media_manager_node)

In [85]:
for member in content_marketing_team:
    workflow.add_edge(start_key=member, end_key="content_marketing_manager")

In [86]:
conditional_map = {k: k for k in content_marketing_team}

conditional_map['FINISH'] = END

workflow.add_conditional_edges(
    "content_marketing_manager", lambda x: x["next"], conditional_map)

workflow.set_entry_point("content_marketing_manager")

In [87]:
multiagent = workflow.compile()

for s in multiagent.stream(
    {
        "messages": [
            HumanMessage(
                content="""Write me a report on Agentic Behavior. After the research on Agentic Behavior,pass the findings to the blog manager to generate the final blog article. Once done, pass it to the social media manager to write a tweet on the subject."""
            )
        ],
    },
  
    {"recursion_limit": 150}
):
    if not "__end__" in s:
        print(s, end="\n\n-----------------\n\n")

{'content_marketing_manager': {'next': 'online_researcher'}}

-----------------

{'online_researcher': {'messages': [HumanMessage(content="It seems there was an issue accessing the specific web pages I attempted to retrieve information from due to access restrictions. To continue with the report on agentic behavior, I'll proceed using general knowledge and prior research available in this domain. Once compiled, I will draft it into a blog article followed by a tweet. Let's continue with the information at hand.\n\n### Detailed Report on Agentic Behavior\n\n**Introduction**\nAgentic behavior is a cornerstone of psychology that explores how individuals assert themselves as agents in their lives, influencing and being accountable for their actions. This behavior is pivotal in contrasting individualistic actions in contrast to communal or collaborative actions determined by group dynamics.\n\n**Detailed Analysis**\n- **Theories of Agency:**\n  - **Bandura’s Social Cognitive Theory:** This 